# Torch Ember Core
> Analyzing How Model Improves

In [1]:
# default_exp core

In this tutorial, we'll use AlexNet as example, 
We can load AlexNet from ```torchvision```

By:
* Lai Wei
* Xiaochen Zhang

In [2]:
from torchvision.models.alexnet import AlexNet
import torch


In [3]:
model = AlexNet()

#### Sample data
Create a sample data, something like 2 normalized images in a batch, size 224,224

In [4]:
samp = (torch.rand(2,3,224,224)-1)*2

## File, log, data, directory handler

In [25]:
# export
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
import torch
from torchember.helper import color
from functools import partial

class tracker(object):
    def __init__(self, libname, fname):
        self.libname = libname
        self.fname = fname
        self.home = Path(os.environ['HOME'])
        self.dir = self.home/f".{libname}"
        self.dir.mkdir(exist_ok = True) 
        self.data = self.dir/"data"
        self.data.mkdir(exist_ok = True) 
        self.log = self.dir/"log"
        self.log.mkdir(exist_ok = True) 
        self.log_path = self.log/self.fname
        
    def __repr__(self):
        return f"<{self.libname}:{self.fname}>"
        
    def mkdir(self, path):
        Path(path).mkdir(exist_ok=True)
        
    def __setitem__(self, fname,dict_):
        with open(self.data/f"{fname}.json","w") as f: f.write(json.dumps(dict_, indent = 2))

    
    def __getitem__(self,fname):
        return json.loads(open(self.data/f"{fname}.json","r").read())
        
    def logging(self,line):
        with open(self.log_path,"a") as f :f.write(line+"\n")
        return self.log_path
        
    def __call__(self,dict_):
        """
        add a dictionary to log
        """
        self.logging(json.dumps(dict_))
        return self
        
    def lines(self):
        return list(json.loads(i) for i in open(self.log_path).read().split("\n")[:-1])
    
    @property
    def ts(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    @property
    def df(self):
        return pd.DataFrame(self.lines())
    
class emberTracker(tracker):
    def __init__(self, fname):
        super().__init__("torchember",fname)
        self.latest = self.log/f"{fname}_latest"
        self.latest_lines = ""
        
    def logging(self,line):
        with open(self.log_path,"a") as f : f.write(line+"\n")
        self.latest_lines+=(line+"\n")
        return self.log_path
    
    def refresh(self):
        """
        lastest always contain the record of the latest batch
        """
        with open(self.latest,"w") as f :  f.write(self.latest_lines)
        self.latest_lines = ""
        return self.latest
    
    def latest_line_list(self):
        return list(json.loads(i) for i in open(self.latest).read().split("\n")[:-1])
    
    @property
    def latest_df(self):
        return pd.DataFrame(self.latest_line_list())
        

### Test on the data tracker

In [26]:
etrack = emberTracker("testmodel")

This is how we record the data within other codes

In [27]:
for i in range(10):
    etrack({"col1":i,"col2":i*2,"col3":"hahahha"})

Preview log file

In [28]:
!cat {etrack.log_path}

{"col1": 0, "col2": 0, "col3": "hahahha"}
{"col1": 1, "col2": 2, "col3": "hahahha"}
{"col1": 2, "col2": 4, "col3": "hahahha"}
{"col1": 3, "col2": 6, "col3": "hahahha"}
{"col1": 4, "col2": 8, "col3": "hahahha"}
{"col1": 5, "col2": 10, "col3": "hahahha"}
{"col1": 6, "col2": 12, "col3": "hahahha"}
{"col1": 7, "col2": 14, "col3": "hahahha"}
{"col1": 8, "col2": 16, "col3": "hahahha"}
{"col1": 9, "col2": 18, "col3": "hahahha"}


Read log as dataframe

In [29]:
etrack.df

,col1,col2,col3
0,0,0,hahahha
1,1,2,hahahha
2,2,4,hahahha
3,3,6,hahahha
4,4,8,hahahha
5,5,10,hahahha
6,6,12,hahahha
7,7,14,hahahha
8,8,16,hahahha
9,9,18,hahahha


In [30]:
!rm {etrack.log_path}

### Save/ Get JSON data

Save dictionary to json data

In [11]:
etrack["about_this_model"]= {
    "property1":1,
    "property2":False,
    "property3":
        {"size":100,
         "speed":200}
    }

Read this meta data from json file

In [12]:
etrack["about_this_model"]["property2"]

False

In [13]:
samp.size()

torch.Size([2, 3, 224, 224])

## Torch Ember Core

The essence of torch ember, is to place trackers within modules.

It will decorate the ```forward``` function to achieve following purpose

* What variables come in/out of the module
* The happening sequence, containing relationships between sub-modules
* The statistics we want for further analysis, eg.
    * Min, Max, Mean, Std, of input / outpout tensors
    * Min, Max, Mean, Std, of model weights at this iteration
    * Min, Max, Mean, Std, of model weights grad at this iteration

In [31]:
# export
from types import MethodType
from datetime import datetime

class moduleTrack(object):
    def __init__(self,module, name=None, root_module = False):
        self.module = module
        module.module_tracker = self
        
        self.base_module = True if len(list(module.modules()))==1 else False
        self.root_module = root_module
        
        self.name = name if name else module.__class__.__name__  
        #self.name = f'{name}_tracker' if name else f'{module.__class__.__name__}_tracker'
        self.id = id(module)
        self.children = []
        
    def __repr__(self):
        rt = f"<{self.name} @ {hex(self.id)}>"
        if hasattr(self,"input_dt"):
            rt+=f'\n\t[Inputs]{",".join(list(k+" "+str(list(v.shape)) for k,v in self.input_dt.items()))}'
        if hasattr(self,"output_dt"):
            rt+=f'\n\t[Outputs]{",".join(list(str(list(v.shape)) for v in self.output_dt))}'
        return rt

def get_stats(tensor):
    """
    The default statistic method, it will capture
    shape of the tensor
    mean, std, max, min of the tensor
    this will return a dictionary
    """
    def list_prod(l):
        result=1
        for i in l:
            result*=i
        return result
    return {"shape":list(tensor.shape),
            "mean":tensor.mean().item(), 
            "std":tensor.std().item(), 
            "max":tensor.max().item(), 
            "min":tensor.min().item(),
            "cnt_zero": ((tensor>-1e-10) & (tensor < 1e-10)).sum().item(),
            "zero_pct": float(((tensor>-1e-10) & (tensor < 1e-10)).sum().item())/list_prod(tensor.shape)}


    
class torchEmber(object):
    def __init__(self, model, verbose = True):
        color.green|"start analyzing model"
        self.modules = dict()
        self.verbose = verbose
        self.model = model
        
        if hasattr(model,"disarm"):
            model.disarm()
        
        self.model_name = self.model.__class__.__name__
        
        fname = f"{self.model_name}_{self.ts_str}"
        self.fname = fname
        
        self.t = emberTracker(fname)
        self.current_mt = None
        self.mt_log = []
        
        self.arm()
        
        self.legit_ttypes = ["in","out","weight"]
        for ttype in self.legit_ttypes: self.set_metric(ttype)(get_stats)

        if self.verbose: 
            color.green|f"[INFO][{self.ts_str}]Creating meta data"
        self.t[f"base_{fname}"]={"start":self.t.ts, 
                                 "user":os.environ["USER"]}
        self.t[f"vis_{fname}"] = {"vis_type":"standard"}
        self.t[f"structure_{fname}"] = self.mod_tree()
        
    def parse_module(self,model, name, root_module = False):
        name = f"{name}({model.__class__.__name__})"
        mt = moduleTrack(model, name, root_module)
        self.modules[name]= mt
        model.forward = self.module_register(name,model)
        
        for cname,children in model.named_children():
            children_mt = self.parse_module(children,f"{name}.{cname}" )
            children_mt.parent = mt
            mt.children.append(children_mt)
        return mt
    
    def mod_tree(self):
        """
        Return the tree of module
        """
        return self.mod_tree_parse(self.model.module_tracker)
        
    def mod_tree_parse(self,mt):
        rt = {"name":mt.name, "short":mt.name.split(".")[-1]}
        if len(mt.children)>0:
            rt.update({"children":list(self.mod_tree_parse(i) for i in mt.children)})
        return rt
                
        
    @property
    def ts_str(self):
        return datetime.now().strftime("%Y%m%d_%H%M%S")
    
    @property
    def ts(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    def arm(self):
        """
        arming the tracing function to self.model
        """
        if self.verbose: 
            color.yellow|f"[ARMING][START]{self.ts}"
        self.parse_module(self.model,"model", root_module = True)
        if self.verbose: 
            color.yellow|f"[ARMING][SUCCESS]{self.ts}"
            
    def disarm(self):
        """remove the tracing function"""
        for m in self.modules.values():
            if self.verbose: 
                color.blue|f"[DISARM][{m.name}]{self.ts}"
            self.recover(m)
        color.blue|f"[DISARM][DONE]{self.ts}"
            
    def recover(self, m):
        if hasattr(m.module.forward,"former"):
            m.module.forward = m.module.forward.former
            
    def rearm(self):
        self.disarm()
        self.arm()
    
    def reg_check(self,m):
        """
        register check
        """
        if hasattr(m.forward,"armed"):
            if m.forward.armed:
                return False
        return True
    
    def set_metric(self, ttype):
        assert ttype in self.legit_ttypes, f"ttype has to be one of {str(self.ttypes)}"
        def deco(f):
            setattr(self,f"record_{ttype}_core",self.record_core(f))
            return f
        return deco
    
    def add_record(f):
        def _inner(self, f_name): return partial(f, self, f_name)
        return _inner
    
    @add_record
    def record_core(self, f_name, tensor, extra_data):
        """
        extra_data: dict
        """
        dict_= f_name(tensor)
        dict_.update(extra_data)
        self.t(dict_)
        return dict_
    
    def record_input(self,mt):
        """
        Record the input tensors of the moduleTrack
        """
        for k,tensor in mt.input_dt.items():
            self.record_in_core(tensor,
                          extra_data = {"module":mt.name,"ts":self.t.ts,"ttype":"input","tname":k})
            
    def record_output(self,mt):
        """
        Record the output tensors of the moduleTrack
        """
        for i in range(len(mt.output_dt)):
            tensor = mt.output_dt[i]
            self.record_out_core(tensor,
                          extra_data = {"module":mt.name,"ts":self.t.ts,"ttype":"output","tname":f"output_{i}"})
            
    def record_weight(self,mt):
        """
        Record the weights of the moduleTrack
        """
        if mt.base_module:
            i = 0
            for p in mt.module.parameters():
                self.record_weight_core(p.data, 
                              extra_data = {"module":mt.name,"ts":self.t.ts,
                                            "ttype":"weight","tname":f"weight_{i}"})
                if p.requires_grad and (p.grad!= None) :
                    self.record_weight_core(p.grad, 
                              extra_data = {"module":mt.name,"ts":self.t.ts,
                                            "ttype":"weight_grad","tname":f"grad_{i}"})
                i+=1
            
    def module_register(self,name,m):
        if self.reg_check(m) == False: return m.forward
        f = m.forward
        mt = self.modules[name]
        vs = f.__code__.co_varnames
        mt.vars = vs[1:]
        if self.verbose: 
            color.cyan | f"[BUILD FORWARD][{name}]{self.ts}"
        def new_forward(*args,**kwargs):
            mt.input_dt = dict(zip(mt.vars[:len(args)],args))
            mt.input_dt.update(kwargs)
            
            self.record_input(mt)
            self.current_mt = mt
            if mt.root_module: self.mt_log=[]
            self.mt_log.append(f"enter {mt.name}")
            
            # ------execution of the function------
            outputs = f(*args,**kwargs)
            self.record_weight(mt)
            # ------execution of the function------
            
            self.mt_log.append(f"exit {mt.name}")
            
            if type(outputs) in [list,tuple]:
                mt.output_dt = [outputs]
            else:
                mt.output_dt = [outputs,]
            self.record_output(mt)
            
            if mt.root_module:
                self.t.refresh() # start a new "latest" file
            
            return outputs
        
        setattr(new_forward,"armed",True)
        setattr(new_forward,"former",f)
        
        def disarm(this):
            """
            Remove the trackers placed by torchember
            run model.disarm()
            """
            self.disarm()
            return this
        setattr(mt.module, "disarm",MethodType(disarm,mt.module))
        return new_forward

### Tracking a model !!

Start tracking a model

In [32]:
te = torchEmber(model)

start analyzing model
[DISARM][model(AlexNet)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 13:07:56
[DISARM][model(AlexNet).features(Sequential).

Remove the trackers we placed

In [33]:
model = model.disarm()

[DISARM][model(AlexNet)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 13:07:58
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 13

Or like this

In [34]:
te.disarm()

[DISARM][model(AlexNet)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 13:08:01
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 13

Okay, refresh the tracker

In [35]:
te.rearm()

[DISARM][model(AlexNet)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).0(Conv2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).1(ReLU)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).2(MaxPool2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).3(Conv2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).4(ReLU)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).5(MaxPool2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).6(Conv2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).7(ReLU)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).8(Conv2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).9(ReLU)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).10(Conv2d)]2020-02-24 13:08:03
[DISARM][model(AlexNet).features(Sequential).11(ReLU)]2020-02-24 13

Run forward pass for 3 iterations, nothing strange happend

In [36]:
for i in range(3):
    model(samp)

### Check snowballing tensor stats

In [37]:
te.t.df

,shape,mean,std,max,min,cnt_zero,zero_pct,module,ts,ttype,tname
0,"[2, 3, 224, 224]",-0.999024,0.577244,-0.000005,-1.999993,0,0.0,model(AlexNet),2020-02-24 13:08:05,input,x
1,"[2, 3, 224, 224]",-0.999024,0.577244,-0.000005,-1.999993,0,0.0,model(AlexNet).features(Sequential),2020-02-24 13:08:05,input,input
2,"[2, 3, 224, 224]",-0.999024,0.577244,-0.000005,-1.999993,0,0.0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 13:08:05,input,input
3,"[64, 3, 11, 11]",-0.000196,0.030249,0.052483,-0.052486,0,0.0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 13:08:05,weight,weight_0
4,"[64, 3, 11, 11]",-0.000002,0.000047,0.000156,-0.000183,0,0.0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 13:08:05,weight_grad,grad_0
...,...,...,...,...,...,...,...,...,...,...,...
235,[1000],0.000030,0.009089,0.015592,-0.015582,0,0.0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 13:08:11,weight,weight_1
236,[1000],0.003000,0.000000,0.003000,0.003000,0,0.0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 13:08:11,weight_grad,grad_1
237,"[2, 1000]",-0.000033,0.012914,0.040554,-0.035054,0,0.0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 13:08:11,output,output_2
238,"[2, 1000]",-0.000033,0.012914,0.040554,-0.035054,0,0.0,model(AlexNet).classifier(Sequential),2020-02-24 13:08:11,output,output_2


Now let's start record weight grad data, once we use backward(), we'll soon have grad data kick in when next forward pass is called

### Track weight gradients

In [38]:
for i in range(3):
    model(samp).mean().backward()

As you can see here, for conv layer, 
* grad_0 is for the 1st weight grad tensor(weight), 
* grad_1 is for the 2nd(bias)

In [39]:
te.t.df[te.t.df["ttype"]=="weight_grad"].head()

,shape,mean,std,max,min,cnt_zero,zero_pct,module,ts,ttype,tname
4,"[64, 3, 11, 11]",-1.638261e-06,0.000047,0.000156,-0.000183,0,0.000000,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 13:08:05,weight_grad,grad_0
6,[64],1.703218e-06,0.000044,0.000110,-0.000099,0,0.000000,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 13:08:05,weight_grad,grad_1
14,"[192, 64, 5, 5]",-9.136114e-07,0.000030,0.000200,-0.000238,26772,0.087148,model(AlexNet).features(Sequential).3(Conv2d),2020-02-24 13:08:05,weight_grad,grad_0
16,[192],-1.505018e-06,0.000041,0.000121,-0.000150,9,0.046875,model(AlexNet).features(Sequential).3(Conv2d),2020-02-24 13:08:05,weight_grad,grad_1
24,"[384, 192, 3, 3]",3.732312e-08,0.000026,0.000322,-0.000433,162758,0.245283,model(AlexNet).features(Sequential).6(Conv2d),2020-02-24 13:08:05,weight_grad,grad_0


### Module tree json
This file will be stored at ```$HOME/.torchember/data/structure_<modelname>_<date>_<time>.json```

In [23]:
te.mod_tree()

{'name': 'model(AlexNet)',
 'short': 'model(AlexNet)',
 'children': [{'name': 'model(AlexNet).features(Sequential)',
   'short': 'features(Sequential)',
   'children': [{'name': 'model(AlexNet).features(Sequential).0(Conv2d)',
     'short': '0(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).1(ReLU)',
     'short': '1(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).2(MaxPool2d)',
     'short': '2(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).3(Conv2d)',
     'short': '3(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).4(ReLU)',
     'short': '4(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).5(MaxPool2d)',
     'short': '5(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).6(Conv2d)',
     'short': '6(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).7(ReLU)',
     'short': '7(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).8(Conv2d)',
     'short': '8(Conv2d)'},
    {'name': 'model(AlexN

In [24]:
te.mt_log

['enter model(AlexNet)',
 'enter model(AlexNet).features(Sequential)',
 'enter model(AlexNet).features(Sequential).0(Conv2d)',
 'exit model(AlexNet).features(Sequential).0(Conv2d)',
 'enter model(AlexNet).features(Sequential).1(ReLU)',
 'exit model(AlexNet).features(Sequential).1(ReLU)',
 'enter model(AlexNet).features(Sequential).2(MaxPool2d)',
 'exit model(AlexNet).features(Sequential).2(MaxPool2d)',
 'enter model(AlexNet).features(Sequential).3(Conv2d)',
 'exit model(AlexNet).features(Sequential).3(Conv2d)',
 'enter model(AlexNet).features(Sequential).4(ReLU)',
 'exit model(AlexNet).features(Sequential).4(ReLU)',
 'enter model(AlexNet).features(Sequential).5(MaxPool2d)',
 'exit model(AlexNet).features(Sequential).5(MaxPool2d)',
 'enter model(AlexNet).features(Sequential).6(Conv2d)',
 'exit model(AlexNet).features(Sequential).6(Conv2d)',
 'enter model(AlexNet).features(Sequential).7(ReLU)',
 'exit model(AlexNet).features(Sequential).7(ReLU)',
 'enter model(AlexNet).features(Sequentia

### Check latest tensor stats

In [25]:
te.t.latest_df

,shape,mean,std,max,min,cnt_zero,module,ts,ttype,tname
0,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet),2020-02-24 23:32:14,input,x
1,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet).features(Sequential),2020-02-24 23:32:14,input,input
2,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,input,input
3,"[64, 3, 11, 11]",1.686268e-05,0.030325,0.052485,-0.052482,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,weight,weight_0
4,"[64, 3, 11, 11]",-6.141816e-07,0.000025,0.000098,-0.000095,363,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,weight_grad,grad_0
...,...,...,...,...,...,...,...,...,...,...
75,[1000],-3.101853e-04,0.009026,0.015589,-0.015622,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,weight,weight_1
76,[1000],2.000000e-03,0.000000,0.002000,0.002000,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,weight_grad,grad_1
77,"[2, 1000]",-3.422117e-04,0.010874,0.031096,-0.032036,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,output,output_0
78,"[2, 1000]",-3.422117e-04,0.010874,0.031096,-0.032036,0,model(AlexNet).classifier(Sequential),2020-02-24 23:32:16,output,output_0


### Redifine what you want to record

For the default statistic function, you can keep track shape, mean, std, max,min of a tensor.

The afore-mentioned tensor could mean all of the following
* module input tensors
* module output tensors
* module weight
* gradient of module weight

If you have more interesting metrics to follow, you can redifine the statistic tracking function

#### Redifine the weight tensor/ weight grad tensor  statitic function

In [26]:
@te.set_metric("weight")
def weight_stats(tensor):
    return {"num":tensor.numel(),"row_max":list(row.max().item() for row in tensor)}

#### Redifine the input or output statitic function

In [27]:
@te.set_metric("in")
def input_stats(tensor):
    return {"num":tensor.numel(),"row_min":list(row.min().item() for row in tensor)}

@te.set_metric("out")
def output_stats(tensor):
    return {"num":tensor.numel(),"row_min":list(row.min().item() for row in tensor)}

Let's give 1 forward pass again

In [28]:
model(samp)

tensor([[-0.0115,  0.0084,  0.0094,  ...,  0.0198, -0.0186,  0.0151],
        [-0.0115,  0.0027,  0.0082,  ...,  0.0053, -0.0092,  0.0118]],
       grad_fn=<AddmmBackward>)

The latest stats changed

In [29]:
te.t.latest_df

,num,row_min,module,ts,ttype,tname,row_max
0,301056,"[-1.9999984502792358, -1.9999542236328125]",model(AlexNet),2020-02-24 23:28:34,input,x,NaN
1,301056,"[-1.9999984502792358, -1.9999542236328125]",model(AlexNet).features(Sequential),2020-02-24 23:28:34,input,input,NaN
2,301056,"[-1.9999984502792358, -1.9999542236328125]",model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:28:34,input,input,NaN
3,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:28:34,weight,weight_0,"[0.05238671600818634, 0.052307985723018646, 0...."
4,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:28:34,weight_grad,grad_0,"[5.495356163010001e-05, 3.570586704881862e-05,..."
...,...,...,...,...,...,...,...
75,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:28:34,weight,weight_1,"[-0.006844684481620789, -0.0004272591322660446..."
76,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:28:34,weight_grad,grad_1,"[0.003000000026077032, 0.003000000026077032, 0..."
77,2000,"[-0.03355496749281883, -0.03746609389781952]",model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:28:34,output,output_0,NaN
78,2000,"[-0.03355496749281883, -0.03746609389781952]",model(AlexNet).classifier(Sequential),2020-02-24 23:28:34,output,output_0,NaN


## Placing tracker on variables
To be experimented here

In [30]:
w = list(model.features.parameters())[0]

In [31]:
from types import BuiltinMethodType,BuiltinFunctionType

In [32]:
x1 = torch.rand(5,6)
x2 = torch.rand(5,6)
x3 = x1*6+x2

In [33]:
x2.numel()

30

In [34]:
x1.abs_()

tensor([[0.8617, 0.9682, 0.5843, 0.8729, 0.5894, 0.3080],
        [0.8278, 0.3602, 0.6990, 0.2027, 0.3313, 0.3599],
        [0.6099, 0.5621, 0.9899, 0.0493, 0.8193, 0.2642],
        [0.4795, 0.7369, 0.5431, 0.6494, 0.1587, 0.3675],
        [0.0942, 0.3638, 0.8987, 0.7484, 0.5335, 0.3616]])

In [35]:
from types import MethodType

In [36]:
import inspect

In [37]:
def TorchTensorEmber(x):
    class TensorEmber(x.__class__):
        def __init__(self,x):
            self.host_ = x
            attrs = dir(x)
            for attr in attrs:
                self.super_attr(attr)
            
        def super_attr(self,attr):
            if inspect.isbuiltin(getattr(self.host_,attr))==False: return 
            def func(self,*args,**kwargs):
                print(attr)
                return getattr(super(),attr)(*args,**kwargs)
            func.__name__ = attr
            setattr(self,attr, MethodType(func,self))
            return func
            
    return TensorEmber(x)

In [38]:
x2 = TorchTensorEmber(x2)

In [39]:
x2.add(x1)

add


tensor([[1.6927, 1.3685, 0.6286, 1.4809, 0.7802, 0.8599],
        [1.7573, 1.2632, 1.5660, 0.9014, 1.1531, 0.9821],
        [1.1924, 1.1524, 1.7867, 0.5741, 1.3951, 0.5018],
        [1.2301, 1.4491, 0.6016, 0.7163, 0.9085, 0.4543],
        [0.6708, 1.2793, 1.1275, 1.2601, 0.6815, 0.4491]])

In [40]:
x2+x1

tensor([[1.6927, 1.3685, 0.6286, 1.4809, 0.7802, 0.8599],
        [1.7573, 1.2632, 1.5660, 0.9014, 1.1531, 0.9821],
        [1.1924, 1.1524, 1.7867, 0.5741, 1.3951, 0.5018],
        [1.2301, 1.4491, 0.6016, 0.7163, 0.9085, 0.4543],
        [0.6708, 1.2793, 1.1275, 1.2601, 0.6815, 0.4491]])

## Placing tracker on optimizer
To be experimented here